# RNN Keras Example
### 周杰伦专辑歌词数据集

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import backend as f
import numpy as np
import math
import sys
import time
import os
lyric_file_path = os.path.join('.','data','jaychou_lyrics.txt.zip')


In [ ]:
# load data function
def load_data_jay_lyrics:
    '''load jay lyrics'''
    with zipfile.ZipFile(lyric_file_path) as zin:
        with zin.open('jaychou_lyrics.txt') as f:
            corpus_chars = f.read.decode('utf-8')
            
            
        